# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [1]:
# Load pickled data
import pickle

training_file = './input/train.p'
validation_file = './input/valid.p'
testing_file = './input/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

# Number of training examples
n_train = len(X_train)

# Number of training examples
n_valid = len(X_valid)

# Number of testing examples.
n_test = len(X_test)

# What's the shape of an traffic sign image?
image_shape = X_train[0].shape

# How many unique classes/labels there are in the dataset.
n_classes = len(set(y_train))

print("Number of training examples =", n_train)
print("Number of validation examples =", n_valid)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc.

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections.

In [ ]:
### Data exploration visualization code goes here.
import random
import itertools
import csv
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline

signnames_file = 'signnames.csv'

signnames = {}

with open(signnames_file, newline='') as f:
    reader = csv.reader(f, delimiter=',', quoting=csv.QUOTE_NONE)
    next(reader, None)  # skip the headers
    for row in reader:
        signnames[int(row[0])] = row[1]

def group_images_by_sign(y_train):
    def sort_sign(index):
        return y_train[index]
    
    sorted_signs = sorted(range(len(y_train)), key=sort_sign)
    return {k: list(v) for k,v in itertools.groupby(sorted_signs, key=sort_sign)}


def display_rand_images_by_sign(X_train, y_train):
    images_by_sign = group_images_by_sign(y_train)
    fig = plt.figure(figsize=(20,50))
    count = 1
    for k, v in images_by_sign.items():
        index = random.randint(0, len(v)-1)
        image = X_train[v[index]].squeeze()
    
        a = fig.add_subplot(11, 4, count)
        a.set_title(signnames[k])
        plt.imshow(image, cmap="gray")
        plt.axis('off')
        count += 1

display_rand_images_by_sign(X_train, y_train)

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
### Preprocess the data here. Preprocessing steps could include normalization, converting to grayscale, etc.

import numpy as np    
from scipy.ndimage.interpolation import rotate, shift
from sklearn.utils import shuffle

# Data augmentation.
def data_augmentation(X_train, y_train):
    def _rotate(x, height, width):
        angle = random.randint(-15, 15)
        return rotate(x, angle, reshape=False)

    def _shift(x, height, width):
        s = np.random.randint(-3,3,3)
        return shift(x, s, order=3, mode='nearest')
    
    def _rotate_and_shift(x, height, width):
        return _rotate(_shift(x, height, width), height, width)
    
    distortion_fcts = (_rotate, _shift, _rotate_and_shift)
    
    new_imgs = []
    labels = []
    height, width, ch = X_train[0].shape
    
    for x, y in zip(X_train, y_train):
        index = random.randint(0, len(distortion_fcts)-1)
        distortion_fct = distortion_fcts[index]
        
        new_img = distortion_fct(x, height, width)
        new_imgs.append(new_img)
        labels.append(y)
        
    new_imgs = np.array(new_imgs)
    labels = np.array(labels)
    
    return new_imgs, labels

X_train_augm, y_train_augm = data_augmentation(X_train, y_train)

# Display some images distorted to increase the data set
def display_modified_images_by_sign(X, y, X_modified):
    images_by_sign = group_images_by_sign(y)
    fig = plt.figure(figsize=(10,300))
    count = 1
    nbcols = 3
    nbrows = len(images_by_sign)
    nbcells = nbcols * nbrows
    
    is_first = True
    
    for k, v in images_by_sign.items():
        index = random.randint(0, len(v)-1)
    
        orig_img = X[v[index]].squeeze()
        dist_img = X_modified[v[index]].squeeze()
        
        c = fig.add_subplot(nbcells, nbcols, count)
        c.text(0,0.5, signnames[k])
        plt.axis('off')
        count += 1
        
        a = fig.add_subplot(nbcells, nbcols, count)
        if is_first:
            a.set_title('before')
        
        plt.imshow(orig_img, cmap="gray")
        plt.axis('off')
        count += 1
        b = fig.add_subplot(nbcells, nbcols, count)
        if is_first:
            b.set_title('after')
            is_first = False
        
        plt.imshow(dist_img, cmap="gray")
        plt.axis('off')
        count += 1

def display_distorted_images_by_sign():
    return display_modified_images_by_sign(X_train, y_train, X_train_augm)
        
display_distorted_images_by_sign()

X_train_augm = np.concatenate((X_train, X_train_augm))
y_train_augm = np.concatenate((y_train, y_train_augm))

X_train, y_train = shuffle(X_train, y_train)
X_train_augm, y_train_augm = shuffle(X_train_augm, y_train_augm)

print("Number of training examples =", len(X_train))
print("Number of augmented training examples =", len(X_train_augm))

### Model Architecture

In [2]:
### Building blocks definition

import tensorflow as tf
from tensorflow.contrib.layers import flatten

def _build_variable(name, shape, initializer):
    return tf.get_variable(name, shape, initializer=initializer, dtype=tf.float32)

def _variable_with_regul_term(name, shape, stddev, wd):
    dtype = tf.float32
    var = _build_variable(name, shape, tf.truncated_normal_initializer(stddev=stddev, dtype=dtype))
    if wd is not None:
        weight_decay = tf.multiply(tf.nn.l2_loss(var), wd)
        tf.add_to_collection('losses', weight_decay)
    return var

def conv_relu(input, kernel_shape, bias_shape, stddev=5e-2, wd=0.0, constant_init=0.0, padding='SAME'):
    # Create variable named "weights".
    weights = _variable_with_regul_term("weights", kernel_shape, stddev=stddev, wd=wd)
    # Create variable named "biases".
    biases = _build_variable("biases", bias_shape, initializer=tf.constant_initializer(constant_init))
    conv = tf.nn.conv2d(input, weights, strides=[1, 1, 1, 1], padding=padding)
    pre_activation = tf.nn.bias_add(conv, biases)
    return tf.nn.relu(pre_activation)

def fully_connected(input, kernel_shape, bias_shape, stddev, wd, constant_init=0.0):
    # Create variable named "weights".
    weights = _variable_with_regul_term("weights", kernel_shape, stddev=stddev, wd=wd)
    # Create variable named "biases".
    biases = _build_variable("biases", bias_shape, initializer=tf.constant_initializer(constant_init))
    matmul = tf.matmul(input, weights)
    return tf.nn.bias_add(matmul, biases)

def relu(input):
    return tf.nn.relu(input)

def max_pool(input, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME'):
    return tf.nn.max_pool(input, ksize=ksize, strides=strides, padding=padding)

def normalize(input, depth_radius=4, bias=1.0, alpha=(0.001 / 9.0), beta=0.75):
    return tf.nn.local_response_normalization(input, depth_radius=depth_radius, bias=bias, alpha=alpha, beta=beta)

def drop_out(input, keep_prob):
    return tf.nn.dropout(input, keep_prob=keep_prob)

#### LeNet

In [3]:
def LeNet(input_queue, num_classes=43):
    pass

#### AlexNet (simplified)

In [4]:
def AlexNetSimplified(input_queue, reuse=None, num_classes=43):
    with tf.variable_scope('conv1', reuse=reuse) as scope:
        conv1 = conv_relu(input_queue, [5, 5, 3, 64], [64])
        
    pool1 = max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1])
    norm1 = normalize(pool1)
    
    with tf.variable_scope('conv2', reuse=reuse) as scope:
        conv2 = conv_relu(norm1, [5, 5, 64, 64], [64], constant_init=0.1)
        
    norm2 = normalize(conv2)
    pool2 = max_pool(norm2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1])
    
    # 
    pool2 = flatten(pool2)
    with tf.variable_scope('local3', reuse=reuse) as scope:
        fc1 = fully_connected(pool2, [64*64, 384], [384], stddev=0.04, wd=0.004)
        fc1 = relu(fc1)
    
    with tf.variable_scope('local4', reuse=reuse) as scope:
        fc2 = fully_connected(fc1, [384, 192], [192], stddev=0.04, wd=0.004)
        fc2 = relu(fc2)
    
    with tf.variable_scope('linear', reuse=reuse) as scope:
        logits = fully_connected(fc2, [192, num_classes], [num_classes], stddev=1/192.0, wd=0.0)
        
    return logits

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [5]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.

def input_pipeline(X, y, batch_size, num_epochs=1000, num_threads=4, capacity=5000, min_after_dequeue=100):
    input_queue = tf.train.slice_input_producer([X, y], num_epochs=num_epochs)
    image_batch, label_batch = tf.train.shuffle_batch(
        [input_queue[0], input_queue[1]],
        batch_size=batch_size,
        num_threads=num_threads,
        capacity=capacity,
        min_after_dequeue=min_after_dequeue,
        enqueue_many=False,
        allow_smaller_final_batch=True)
    
    image_batch = tf.cast(image_batch, tf.float32)
    label_batch = tf.cast(label_batch, tf.float32)
    return image_batch, label_batch

def loss(logits, labels):
    labels = tf.cast(labels, tf.int64)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    # Add the losses from the regularization term
    tf.add_to_collection('losses', cross_entropy_mean)
    return tf.add_n(tf.get_collection('losses'))

def evaluate(logits, labels, num_classes=43):
    labels = tf.cast(labels, tf.int64)
    one_hot_y = tf.one_hot(labels, num_classes)
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

def backpropagate(total_loss, global_step, init_learning_rate=0.001):
    optimizer = tf.train.AdamOptimizer(learning_rate=init_learning_rate)
    return optimizer.minimize(total_loss, global_step=global_step)

In [6]:
import time

X_train, y_train = X_train[0:1000], y_train[0:1000]
X_valid, y_valid = X_valid[0:1000], y_valid[0:1000]
X_test, y_test = X_test[0:1000], y_test[0:1000]

def _build_checkpoint_basename(neural_network_fct, batch_size):
    return '%s_%s_model' % (neural_network_fct.__name__, batch_size)

def get_save_path(neural_network_fct, batch_size):
    return '/output/%s' % (_build_checkpoint_basename(neural_network_fct, batch_size))

def _build_accuracy_check(X, y, neural_network_fct, reuse=True):
    # Define the pipeline for the validation/test
    X_ph = tf.placeholder(tf.float32, (None, 32, 32, None))
    y_ph = tf.placeholder(tf.int32, (None))
    digits = neural_network_fct(X_ph, reuse=reuse)
    accuracy = evaluate(digits, y_ph)
    return accuracy, {X_ph: X, y_ph: y}

def train(checkpoint_dir, X_train, y_train, X_valid, y_valid, X_test, y_test, neural_network_fct, batch_size=8, num_epochs=100000, last_step=200, log_frequency = 10): 
    import os
    
    global_step = tf.contrib.framework.get_or_create_global_step()
        
    # Define the pipeline for the training
    images_train, labels_train = input_pipeline(X_train, y_train, batch_size, num_epochs=num_epochs)
    digits_train = neural_network_fct(images_train)
    loss_train = loss(digits_train, labels_train)
    accuracy_train = evaluate(digits_train, labels_train)
            
    # Define the pipeline for the validation
    accuracy_valid, feed_dict_valid = _build_accuracy_check(X_valid, y_valid, neural_network_fct)
        
    # Define the pipeline for the test set (generalization)
    accuracy_test, feed_dict_test = _build_accuracy_check(X_test, y_test, neural_network_fct)
            
    train_op = backpropagate(loss_train, global_step)
    
    class _LoggerHook(tf.train.SessionRunHook):
        """Logs loss and runtime."""

        def begin(self):
            self._step = -1
            self._start_time = time.time()
                
        def end(self, session):
            acc_test = session.run(accuracy_test, feed_dict=feed_dict_test)
            print("Test Accuracy = {:.3f}".format(acc_test))
            
        def before_run(self, run_context):
            self._step += 1
            return tf.train.SessionRunArgs([loss_train,accuracy_train,accuracy_valid], feed_dict=feed_dict_valid)  # Asks for loss value.

        def after_run(self, run_context, run_values):
            if self._step % log_frequency == 0:
                current_time = time.time()
                duration = current_time - self._start_time
                self._start_time = current_time

                loss_value = run_values.results[0]
                acc_train = run_values.results[1]
                acc_valid = run_values.results[2]
                examples_per_sec = log_frequency * batch_size / duration
                sec_per_batch = float(duration / log_frequency)

                format_str = ('step %d, loss = %.2f, acc = %.2f/%.2f (%.1f examples/sec; %.3f sec/batch)')
                print (format_str % (self._step, loss_value, acc_train, acc_valid, examples_per_sec, sec_per_batch))
                 
    saver = tf.train.Saver()
    checkpoint_basename = _build_checkpoint_basename(neural_network_fct, batch_size)
                
    with tf.train.MonitoredTrainingSession(
            save_checkpoint_secs=None,
            save_summaries_steps=None,
            hooks=[tf.train.CheckpointSaverHook(checkpoint_dir=checkpoint_dir, checkpoint_basename=checkpoint_basename, save_steps=100, saver=saver),
                    tf.train.StopAtStepHook(last_step=last_step),
                    tf.train.NanTensorHook(loss_train),
                    _LoggerHook()],
            config=tf.ConfigProto(
                log_device_placement=False)) as mon_sess:
      
        while not mon_sess.should_stop():
            mon_sess.run(train_op)
        
def model_accuracy(checkpoint_dir, X_test, y_test, neural_network_fct, batch_size):
    accuracy_test, feed_dict_test = _build_accuracy_check(X_test, y_test, neural_network_fct, reuse=True)
    saver = tf.train.Saver()
    
    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    
    with tf.Session() as sess:
        save_path = ckpt.model_checkpoint_path
        saver.restore(sess, save_path)
        
        acc_test = sess.run(accuracy_test, feed_dict=feed_dict_test)
        print("Test Accuracy = {:.3f}".format(acc_test))
        
def build_train_accuracy_AlexNetSimplified_fcts(checkpoint_dir="./output/", batch_size=256, num_epochs=100000):
    def train_AlexNetSimplified(batch_size=batch_size, num_epochs=num_epochs):
        last_step = 1000000 #int(n_train/batch_size)+1
        print('Last step: %s'%(last_step))
        print('Num epochs: %s'%(num_epochs))
        return train(checkpoint_dir, X_train, y_train, X_valid, y_valid, X_test, y_test, AlexNetSimplified, batch_size, num_epochs, last_step)

    def accuracy_AlexNetSimplified():
        model_accuracy(checkpoint_dir, X_test, y_test, AlexNetSimplified, batch_size)
        
    return train_AlexNetSimplified, accuracy_AlexNetSimplified
        
train_1, accuracy_1 = build_train_accuracy_AlexNetSimplified_fcts(num_epochs=1)

%time train_1(num_epochs=1)
%time accuracy_1()



Last step: 1000000
Num epochs: 1
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./output/AlexNetSimplified_256_model.
step 0, loss = 7.85, acc = 0.00/0.09 (348.3 examples/sec; 0.735 sec/batch)
INFO:tensorflow:Saving checkpoints for 4 into ./output/AlexNetSimplified_256_model.
Test Accuracy = 0.018
CPU times: user 1min 22s, sys: 13.8 s, total: 1min 36s
Wall time: 21 s
Test Accuracy = 0.018
CPU times: user 9.18 s, sys: 2.04 s, total: 11.2 s
Wall time: 2.24 s


In [7]:
# Evaluate
def validation_set():
    return tf.constant(X_valid, dtype=tf.float32), tf.constant(y_valid, dtype=tf.int64)


# Define the pipeline for the validation
# images_valid, labels_valid = validation_set()
#       digits_valid = neural_network_fct(images_valid, reuse=True)
#       accuracy_valid = evaluate(digits_valid, labels_valid)

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [8]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Predict the Sign Type for Each Image

In [9]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

### Analyze Performance

In [10]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tk.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [11]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

---

## Step 4: Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [12]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it maybe having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")

### Question 9

Discuss how you used the visual output of your trained network's feature maps to show that it had learned to look for interesting characteristics in traffic sign images


**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 